In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import requests
import json

In [2]:
# read in csv files

per_aux_df = pd.read_csv('Data/PER_AUX.csv', encoding='cp1252')

In [3]:

per_aux_df.head()


,ST_CASE,A_AGE1,A_AGE1_FORMAT,A_AGE2,A_AGE2_FORMAT,A_AGE3,A_AGE3_FORMAT,A_AGE4,A_AGE4_FORMAT,A_AGE5,...,A_PTYPE,A_PTYPE_FORMAT,A_RCAT,A_RCAT_FORMAT,A_RESTUSE,A_RESTUSE_FORMAT,YEAR,YEAR_FORMAT,VEH_NO,PER_NO
0,10001,3,25-54,4,25-34,7,25-34,4,25-34,4,...,1,Driver,0,Not Applicable (Coded for Fatalities Only),1,Restrained,2019,2019,1,1
1,10001,3,25-54,5,35+,9,45-54,6,45-64,6,...,2,Occupant,1,White,1,Restrained,2019,2019,1,2
2,10001,4,55+,5,35+,10,55-64,6,45-64,7,...,1,Driver,0,Not Applicable (Coded for Fatalities Only),1,Restrained,2019,2019,2,1
3,10002,3,25-54,5,35+,8,35-44,5,35-44,5,...,1,Driver,1,White,3,Unknown,2019,2019,1,1
4,10002,3,25-54,5,35+,9,45-54,6,45-64,6,...,1,Driver,0,Not Applicable (Coded for Fatalities Only),1,Restrained,2019,2019,2,1


In [4]:
per_aux_df.describe()

dc_per_aux_df = per_aux_df.filter(['ST_CASE','A_AGE3','A_AGE3_FORMAT', 'A_PTYPE_FORMAT', 'A_DOA_FORMAT'], axis=1)
dc_per_aux_df


,ST_CASE,A_AGE3,A_AGE3_FORMAT,A_PTYPE_FORMAT,A_DOA_FORMAT
0,10001,7,25-34,Driver,Not a Fatality
1,10001,9,45-54,Occupant,Died at Scene of Crash
2,10001,10,55-64,Driver,Not a Fatality
3,10002,8,35-44,Driver,Died at Scene of Crash
4,10002,9,45-54,Driver,Not a Fatality
...,...,...,...,...,...
82215,560120,7,25-34,Driver,Died at Scene of Crash
82216,560120,6,21-24,Occupant,Not a Fatality
82217,560121,8,35-44,Driver,Died at Scene of Crash
82218,560121,8,35-44,Occupant,Died at Scene of Crash


In [5]:
na_per_aux_df = dc_per_aux_df[~dc_per_aux_df.isin(["NOT APPLICABLE", "Not Reported", "Unknown", "Other"]).any(axis=1)]
driver_per_aux = na_per_aux_df.loc[na_per_aux_df['A_PTYPE_FORMAT'] == "Driver"]
driver_per_aux["A_AGE3_FORMAT"].value_counts()

25-34    10507
35-44     8301
45-54     7532
55-64     7166
21-24     4590
65-74     4404
16-20     3892
75+       3229
13-15      123
8-12        12
4-7          2
Name: A_AGE3_FORMAT, dtype: int64

In [6]:
mod_person_df= driver_per_aux.replace(
    {"Died at Scene of Crash": "Died", "Died at Other Location": "Died", "Died en Route to a Hospital or Treatment Facility": "Died", "Died at Unknown Location": "Died"})

In [7]:
cleaned_person_df = mod_person_df.rename(columns={"ST_CASE": "Case_Number", "A_AGE3_FORMAT": "Age_Group", "A_PTYPE_FORMAT": "Passenger_Type",
                                      "A_DOA_FORMAT": "DOA_Status"})

In [8]:
grouped_per = cleaned_person_df.groupby(['Age_Group'])
grouped_per["Age_Group"].value_counts()

Age_Group  Age_Group
13-15      13-15          123
16-20      16-20         3892
21-24      21-24         4590
25-34      25-34        10507
35-44      35-44         8301
4-7        4-7              2
45-54      45-54         7532
55-64      55-64         7166
65-74      65-74         4404
75+        75+           3229
8-12       8-12            12
Name: Age_Group, dtype: int64

In [9]:
person_df = pd.read_csv('Data/Person_FARS.csv', encoding='cp1252')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
person_df.head(10)

pers_df = person_df.filter(['STATENAME','ST_CASE','MONTHNAME', 'MAKENAME', 'AGE', 'SEXNAME', 'DOANAME', 'PER_TYPNAME'], axis=1)
pers_df

,STATENAME,ST_CASE,MONTHNAME,MAKENAME,AGE,SEXNAME,DOANAME,PER_TYPNAME
0,Alabama,10001,February,Toyota,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
1,Alabama,10001,February,Toyota,53,Male,Died at Scene,Passenger of a Motor Vehicle In-Transport
2,Alabama,10001,February,Freightliner,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,Alabama,10002,January,BMW,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,Alabama,10002,January,Chevrolet,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
...,...,...,...,...,...,...,...,...
82215,Wyoming,560120,September,Hyundai,31,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82216,Wyoming,560120,September,Hyundai,22,Male,Not Applicable,Passenger of a Motor Vehicle In-Transport
82217,Wyoming,560121,September,Dodge,42,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82218,Wyoming,560121,September,Dodge,41,Female,Died at Scene,Passenger of a Motor Vehicle In-Transport


In [11]:
na_pers = pers_df[~pers_df.isin(["NOT APPLICABLE", "Not Reported", "Unknown", "Other", 999, 998]).any(axis=1)]
only_drivers = na_pers.loc[na_pers['PER_TYPNAME'] == "Driver of a Motor Vehicle In-Transport"]
only_drivers["PER_TYPNAME"].value_counts()

Driver of a Motor Vehicle In-Transport    49664
Name: PER_TYPNAME, dtype: int64

In [12]:
only_drivers.head()

,STATENAME,ST_CASE,MONTHNAME,MAKENAME,AGE,SEXNAME,DOANAME,PER_TYPNAME
0,Alabama,10001,February,Toyota,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
2,Alabama,10001,February,Freightliner,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,Alabama,10002,January,BMW,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,Alabama,10002,January,Chevrolet,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
5,Alabama,10003,January,Acura,22,Male,Not Applicable,Driver of a Motor Vehicle In-Transport


In [13]:
cleaned_pers_df = only_drivers.rename(columns={"STATENAME": "State", "MONTHNAME": "Month",
                                      "MAKENAME": "Vehicle_Model", "SEXNAME": "Sex", "DOANAME": "DOA_Status", "PER_TYPNAME": "Passenger_Type"})
cleaned_pers_df

,State,ST_CASE,Month,Vehicle_Model,AGE,Sex,DOA_Status,Passenger_Type
0,Alabama,10001,February,Toyota,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
2,Alabama,10001,February,Freightliner,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
3,Alabama,10002,January,BMW,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport
4,Alabama,10002,January,Chevrolet,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport
5,Alabama,10003,January,Acura,22,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
...,...,...,...,...,...,...,...,...
82213,Wyoming,560118,September,Peterbilt,56,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
82214,Wyoming,560119,September,Harley-Davidson,52,Male,Not Applicable,Driver of a Motor Vehicle In-Transport
82215,Wyoming,560120,September,Hyundai,31,Male,Died at Scene,Driver of a Motor Vehicle In-Transport
82217,Wyoming,560121,September,Dodge,42,Male,Died at Scene,Driver of a Motor Vehicle In-Transport


In [14]:
# Establish bins for ages
bins = cleaned_pers_df['Age_Group'] = pd.qcut(cleaned_pers_df['AGE'],
                          q=7,duplicates='drop')

# Create labels for these bins
age_group =  ["0 to 15", "16-24", "25-34", "35-44", "45-54", "55-64",
                "65+"]   



In [15]:
cleaned_pers_df

,State,ST_CASE,Month,Vehicle_Model,AGE,Sex,DOA_Status,Passenger_Type,Age_Group
0,Alabama,10001,February,Toyota,34,Female,Not Applicable,Driver of a Motor Vehicle In-Transport,"(29.0, 37.0]"
2,Alabama,10001,February,Freightliner,59,Male,Not Applicable,Driver of a Motor Vehicle In-Transport,"(55.0, 65.0]"
3,Alabama,10002,January,BMW,42,Female,Died at Scene,Driver of a Motor Vehicle In-Transport,"(37.0, 46.0]"
4,Alabama,10002,January,Chevrolet,54,Female,Not Applicable,Driver of a Motor Vehicle In-Transport,"(46.0, 55.0]"
5,Alabama,10003,January,Acura,22,Male,Not Applicable,Driver of a Motor Vehicle In-Transport,"(5.999, 23.0]"
...,...,...,...,...,...,...,...,...,...
82213,Wyoming,560118,September,Peterbilt,56,Male,Not Applicable,Driver of a Motor Vehicle In-Transport,"(55.0, 65.0]"
82214,Wyoming,560119,September,Harley-Davidson,52,Male,Not Applicable,Driver of a Motor Vehicle In-Transport,"(46.0, 55.0]"
82215,Wyoming,560120,September,Hyundai,31,Male,Died at Scene,Driver of a Motor Vehicle In-Transport,"(29.0, 37.0]"
82217,Wyoming,560121,September,Dodge,42,Male,Died at Scene,Driver of a Motor Vehicle In-Transport,"(37.0, 46.0]"
